In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.python.keras.callbacks import ModelCheckpoint

In [ ]:
train_data_dir = ""
val_data_dir = 
test_data_dir = ""
labels = ""

img_height = 150
img_width = 150
batch_size 128

In [ ]:
conv_base = tf.keras.applications.vgg16.VGG16(include_top=False,
                                              weights="imagenet",
                                              pooling=None,
                                              input_shape=(img_width, img_height, 3)
                                             )

In [ ]:
for layer in conv_base.layers[0:]:
    layer.trainable = False

In [ ]:
for i, layer in enumerate(conv_base.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
inputs = tf.keras.Input(shape=(150, 150, 3))
x = keras.applications.vgg16.preprocess_input(inputs)
x = conv_base(x, training=False)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(1012, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.15)(x)
outputs = keras.layers.Dense(6, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
#Number of images to load at each iteration
batch_size = 128

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(zoom_range=0.2,
                                                                brightness_range = (0.8,1.2),
                                                                validation_split=0.25
                                                               )

test_datagen =  tf.keras.preprocessing.image.ImageDataGenerator()

print('Total number of images for "training":')
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_height, img_width),
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical',
                                                    subset='training')
print('Total number of images for "validation":')
val_generator = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size = (img_height, img_width),
                                                 batch_size = batch_size,
                                                 class_mode = "categorical",
                                                 shuffle=False,
                                                 subset='validation')
print('Total number of images for "testing":')
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size = (img_height, img_width),
                                                  batch_size = batch_size,
                                                  shuffle=False,
                                                  class_mode='categorical'
                                                  )

In [ ]:
checkpoint = ModelCheckpoint("OnlyOutputL.h5", 
                             monitor = 'val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', period=1)

In [ ]:
model.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(lr=0.0001, clipnorm = 1.), 
              metrics = ['acc'])

In [ ]:
history = model.fit(train_generator, 
                    epochs=10, 
                    shuffle=True, 
                    validation_data=val_generator,
                    callbacks=[checkpoint]
                    )

In [ ]:
conv_base.trainable = True
print("Number of layers in the base model: ", len(conv_base.layers))

In [ ]:
fine_tune_at = 11

for layer in conv_base.layers[:fine_tune_at]:
    layer.trainable =  False
for i, layer in enumerate(conv_base.layers):
    print(i, layer.name, layer.trainable)    

In [ ]:
model.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(lr=0.00001, clipnorm = 1.), 
              metrics = ['acc'])

In [ ]:
history_fine = model.fit(train_generator, 
                         shuffle=True, 
                         epochs=10,
                         validation_data=test_generator,
                        )

In [ ]:
model.evaluate(test_generator)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)

plt.figure()
plt.plot(epochs, acc, 'b', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

In [ ]:
url_image = ""
url_path = tf.keras.utils.get_file('', origin=url_image)

pic = tf.keras.preprocessing.image.load_img(url_path, target_size=(150,150))
pic = tf.keras.preprocessing.image.img_to_array(pic)
pic = tf.expand_dims(pic, 0)
prediction = model.predict(pic)
print('Label: ', labels[np.argmax(prediction)]) 
print('Confidence: ', 100 * np.max(prediction))